In [1]:
import pandas as pd #DataFrame yapısı oluşturmak için
from bs4 import BeautifulSoup # Web Scraping yapmak için
import requests
from tqdm import tqdm # Döngüleri izlemek için

In [2]:
url = "https://www.imdb.com/list/ls068082370/" #Resmi imdb sayfasının url'sini url değişkenine atadık

In [3]:
def all_page_link(start_url): #Saklamak istediğimiz tüm urlleri bize bulan fonksiyon
    all_urls = []
    url = start_url
    while(url != None):            #Gerekli tüm web sayfaları ara ve son sayfaya gelince döngüden çık
        all_urls.append(url)
        soup = BeautifulSoup(requests.get(url).text,"html.parser")
        next_links = soup.find_all(class_='flat-button lister-page-next next-page')    #Sonraki sayfa bağlantısını çıkartır
        if (len(next_links) == 0):         # Bir sonraki sayfa değilse 0 döndür
            url = None
        else:
            next_page = "https://www.imdb.com" + next_links[0].get('href')
            url = next_page
    return all_urls

In [4]:
# Bu fonksiyonu url den veri ayıklama esnasında aynı satırda bulundukları için director ve actor'u ayırmak için kullandım
def director_and_star_converter(Director_and_star):
    Director_and_star =  Director_and_star.replace("\n","")
    Director_and_star = Director_and_star.replace("|","")
    Director_and_star = Director_and_star.split("Stars:")
    Director_and_star[0] = Director_and_star[0].replace("Director:","")
    Director_and_star[0] = Director_and_star[0].replace("Directors:","")
    for i in range(10):
        Director_and_star[0]=Director_and_star[0].replace("  "," ")
    director = Director_and_star[0][1:]
    stars = Director_and_star[1]
    stars = stars.replace(":","")
    return director,stars

In [6]:
# Yine aynı satırda bulundukları için ayrıştırma fonksiyonu kullanıyoruz.
def votes_and_gross_converter(votes_and_gross):
    votes_and_gross_list = []
    for i in votes_and_gross:
        votes_and_gross_list.append(i.text)
    if(len(votes_and_gross)==2):
        votes=votes_and_gross_list[0]
        gross = votes_and_gross_list[1]
    else:
        votes=votes_and_gross_list[0]
        gross = None
    
    return votes,gross

In [43]:
# Burada tüm ana işlemleri yapıyoruz.
main_array = [] # Tüm verileri saklayan dizi
for url in tqdm(all_page_link("https://www.imdb.com/list/ls068082370/")):     # Tüm sayfalar için fonksiyonu çalıştırır. Aynı zamanda tqdm sayesinde progress barda oluşturur
    soup = BeautifulSoup(requests.get(url).text,"html.parser")         # Ana html kodunu çıkartır.
    for link in soup.find_all(class_='lister-item-content'):
        Rank = int(link.find('span',{"class":"lister-item-index unbold text-primary"}).text[:-1]) # Filmin id'sini saklar
        Title = link.find('a').text # Filmin adını saklar
        Year = link.find('span',{"class":"lister-item-year text-muted unbold"}).text # Filmin çıkış tarihini saklar.
        Runtime = int(link.find('span',{"class":"runtime"}).text[:-4]) # Filmin uzunluğunu saklar.
        Genre = link.find('span',{"class":"genre"}).text[1:] # Filmin türünü saklar.
        Rating = float(link.find('span',{"class":"ipl-rating-star__rating"}).text) # Filmin imdb puanını saklar.
        About = link.find_all('p')[1].text[5:] # Film hakkında kısa bir açıklama depolar
        Metascore = link.find('span', class_='metascore').text if link.find('span', class_='metascore') else '-' #Filmin metascore puanını saklar.
        Director,Actors = director_and_star_converter(link.find_all('p',{"class":"text-muted text-small"})[1].text) # Yönetmen ve oyuncuların adını saklar.
        Votes, Gross = votes_and_gross_converter(link.find_all('span',{"name":"nv"})) # Filmin oy sayısını ve gelirini saklar.
        Votes = int(Votes.replace(",","")) # Oylar verisini int veri tipine dönüştürür.
        Metascore = int(Metascore.replace("-","50")) # Metascore verisini int veri tipine dönüştürür.
        list_of_all = [Rank,Title,Year,Runtime,Genre,Rating,About,Director,Actors,Votes,Gross,Metascore] # Bütün verileri list_of_all değişkenini aktarır.
        main_array.append(list_of_all) # list_of_all u ana dizimize ekledik.

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:13<00:00,  4.35s/it]


In [50]:
# index değişkeni dataframe in sütun adlarını tanımlar.
index = ["Rank","Title","Year","Runtime","Genre","Rating","About","Director","Actors","Votes","Gross","Metascore"]

In [51]:
df = pd.DataFrame(main_array,columns=index)   # main_array'da sakladığım veriler ile bir dataframe oluşturdum


In [52]:
df['Year'] = df['Year'].str.extract('(\d+)').astype(int) # Year değişkeninde ki fazlalıkları atıp int veri tipine dönüştürüyor.
 
df['Gross'] = df['Gross'].astype(float) # Gross değişkenini float veri tipine dönüştürüyor.

df.head(5)

,Rank,Title,Year,Runtime,Genre,Rating,About,Director,Actors,Votes,Gross,Metascore
0,1,The Shawshank Redemption,1994,142,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",2261592,28.34,80
1,2,The Godfather,1972,175,"Crime, Drama",9.2,The aging patriarch of an organized crime dyna...,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",1560378,134.97,100
2,3,The Dark Knight,2008,152,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2226619,534.86,84
3,4,The Godfather: Part II,1974,202,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall, Dian...",1090504,57.30,90
4,5,Pulp Fiction,1994,154,"Crime, Drama",8.9,"The lives of two mob hitmen, a boxer, a gangst...",Quentin Tarantino,"John Travolta, Uma Thurman, Samuel L. Jackson,...",1769128,107.93,94


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 12 columns):
Rank         250 non-null int64
Title        250 non-null object
Year         250 non-null int32
Runtime      250 non-null int64
Genre        250 non-null object
Rating       250 non-null float64
About        250 non-null object
Director     250 non-null object
Actors       250 non-null object
Votes        250 non-null int64
Gross        224 non-null float64
Metascore    250 non-null int64
dtypes: float64(2), int32(1), int64(4), object(5)
memory usage: 22.6+ KB


In [55]:
    df.to_csv("imdb_top250.csv")